# Info from the web

**This notebook goes with [a blog post at Agile*](http://ageo.co/xlines02).**

We're going to get some info from a web service, and from Wikipedia. We'll make good use of [the `requests` library](http://docs.python-requests.org/en/master/), a really nicely designed Python library for making web requests in Python.

## Simple http server

Before we do anything else, let's have a look at running a very simple server right on your computer.

Let's say you want to share a file with someone on your local network.

You can use Python's [`http.server`](https://docs.python.org/3/library/http.server.html) package for this. 

### 1. Open a terminal

Then change directory to the location from which you want to share files.

### 2. Find your IP address

Your local IP address is given by `ipconfig` on Windows, or `ifconfig` on Mac or Linux (you might need to install something first).

### 3. Start server

Type:

    python -m http.server
    
(Note the use of the `-m` switch to tell Python to run a module in its path directly.)

### 4. Visit the IP address

Visit the IP address on another device on the same network. Don't forget to add the port, usually `8000`, to the end of the IP address.

**You can see the files on the host computer!**

## Using the curvenam.es web API

[`curvenam.es`](http://curvenam.es) is a little web app for looking up curve mnemonics from LAS files.

Here's what [the demo request from the site](http://curvenam.es/lookup) looks like:

    http://curvenam.es/lookup?mnemonic=TEST&method=fuzzy&limit=5&maxdist=2
    
We split this into the URL, and the query parameters:

In [ ]:
import requests

In [ ]:
url = 'http://curvenam.es/lookup'

In [ ]:
params = {'mnemonic': 'DT4P',
          'method': 'fuzzy',
          'limit': 1,
          'maxdist': 2
         }

In [ ]:
r = requests.get(url, params)

If we were successful, the server sends back a `200` status code:

In [ ]:
r.status_code

In [ ]:
r.headers

The result of the query is in the `text` attribute of the result:

In [ ]:
r.text

There's a convenient `json()` method to give us the result as JSON:

In [ ]:
r.json()

In [ ]:
try:
    print(r.json()['result'][0]['curve']['description'])
except:
    print("No results")

----

## Scraping geological ages from Wikipedia

Sometimes there isn't a nice API and we have to get what we want from unstructured data. Let's use the task of getting geological ages from Wikipedia as an example.

We'll start with the Jurassic, then generalize.

In [ ]:
url = "http://en.wikipedia.org/wiki/Jurassic"

I used `View Source` in my browser to figure out where the age range is on the page, and what it looks like. The most predictable spot, that will work on every period's page, is in the infobox. It's given as a range, in italic text, with "million years ago" right after it.

Try to find the same string here.

In [ ]:
r = requests.get(url)

Now we have the entire text of the webpage, along with some metadata. The text is stored in `r.text`, and I happen to know that the piece of text we need contains "million years ago":

In [ ]:
r.text.find('million years ago')

In [ ]:
r.text[8800:8900]

We can get at that bit of text using a [regular expression](https://docs.python.org/2/library/re.html):

In [ ]:
import re

s = re.search(r'<i>(.+?million years ago)</i>', r.text)
text = s.group(1)
text

And if we're really cunning, we can get the start and end ages:

In [ ]:
start, end = re.search(r'<i>([\.0-9]+)–([\.0-9]+)&#160;million years ago</i>', r.text).groups()
duration = float(start) - float(end)

print("According to Wikipedia, the Jurassic lasted {:.2f} Ma.".format(duration))

<div class="alert alert-success">
<h3>Exercise</h3>

- Make a function to get the start and end ages of *any* geologic period, taking the name of the period as an argument.
</div>

You should be able to call your function like this:

In [ ]:
get_age('Cretaceous')

<div class="alert alert-success">
<h3>Exercise</h3>

- Make a function that returns the sentence we made before, with the duration, calling the function you just wrote:
</div>

In [ ]:
duration('Cretaceous')

<hr />

<div>
<img src="https://avatars1.githubusercontent.com/u/1692321?s=50"><p style="text-align:center">© Agile Geoscience 2016</p>
</div>